*** UNDER CONSTRUCTION ***


# HTLC-Timeout and HTLC-Success Transactions
## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'
- Lightning Network BOLT #3: '[Commitment Transactions](https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction)'
- Finite Fields, Elliptic Curves and Serialization e.g. [Programming Bitcoin](https://github.com/jimmysong/programmingbitcoin), in particular [Chapter 1](https://github.com/jimmysong/programmingbitcoin/blob/master/ch01.asciidoc), [Chapter2](https://github.com/jimmysong/programmingbitcoin/blob/master/ch02.asciidoc), [Chapter 3](https://github.com/jimmysong/programmingbitcoin/blob/master/ch03.asciidoc) and [Chapter 4](https://github.com/jimmysong/programmingbitcoin/blob/master/ch04.asciidoc).

## HTLC-Timeout and HTLC-Success Transactions

The HTLC-Timeout and HTLC-Success Transactions are almost identical, except the HTLC-timeout transaction is timelocked. Both HTLC-timeout/HTLC-success transactions can be spent by a valid penalty transaction.

* version: 2
* locktime: 0 for HTLC-success, cltv_expiry for HTLC-timeout
* txin count: 1
    * txin[0] outpoint: txid of the commitment transaction and output_index of the matching HTLC output for the HTLC transaction
    * txin[0] sequence: 0 (set to 1 for option_anchors)
    * txin[0] script bytes: 0
    * txin[0] witness stack: 0 <remotehtlcsig> <localhtlcsig>  <payment_preimage> for HTLC-success, 0 <remotehtlcsig> <localhtlcsig> <> for HTLC-timeout

* txout count: 1
    * txout[0] amount: the HTLC amount_msat divided by 1000 (rounding down) minus fees in satoshis (see Fee Calculation)
    * txout[0] script: version-0 P2WSH with witness script as shown below

if option_anchors applies to this commitment transaction, SIGHASH_SINGLE|SIGHASH_ANYONECANPAY is used as described in [BOLT #5](https://github.com/lightning/bolts/blob/master/05-onchain.md#generation-of-htlc-transactions).

The witness script for the output is:

```
OP_IF
    # Penalty transaction
    <revocationpubkey>
OP_ELSE
    `to_self_delay`
    OP_CHECKSEQUENCEVERIFY
    OP_DROP
    <local_delayedpubkey>
OP_ENDIF
OP_CHECKSIG
```


## Setup
### Requirements

For this exercise we'll run the previous notebook to create the commitment transactions beetween with offer and received HTLC.

**You'll need to edit these next line for your local setup.**

In [1]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 4 - Unilateral Closing Transaction/Unilateral Closing Transaction.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd9

### Creating the HTLC-Timeout and HTLC-Success Transactions

Now that the commitment transaction, containing HTLC outputs, was published forcing close the channel, we can create the HTLC-Timeout and/or HTLC-Success Transactions to reclaim the funds locked on HTLCs.

Let's start with simulating an HTLC Timeout transaction.

In [2]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER
marker = bytes.fromhex("00")

# FLAG
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(alice_signed_commitment_txid))[::-1]
index = bytes.fromhex("0200 0000")

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: set to 1 for option_anchors
sequence = bytes.fromhex("0100 0000")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
output_count = bytes.fromhex("01")

# OP_IF
#   Penalty transaction
#   <revocationpubkey>
# OP_ELSE
#    `to_self_delay`
#    OP_CHECKSEQUENCEVERIFY
#    OP_DROP
#    <local_delayedpubkey>
# OP_ENDIF
# OP_CHECKSIG
htlc_redeemScript = bytes.fromhex(
    "63"   # OP_IF
    + "21"
    + bob_revocation_pubkey
    + "67"   # OP_ELSE
    + "02"   
    + "9000" # to_self_delay of 144 blocks
    + "b2"   # OP_CHECKSEQUENCEVERIFY
    + "75"   # OP_DROP
    + "21"
    + alice_delayed_pubkey
    + "68"  # OP_ENDIF
    + "ac") # OP_CHECKSIG

htlc_script_hash = sha256(htlc_redeemScript)
htlc_output_spk = bytes.fromhex("0020") + htlc_script_hash
print(f"HTLC Output SPK: {htlc_output_spk.hex()}")

outputs = (
    alice_offered_htlc_output_value
    + varint_len(htlc_output_spk)
    + htlc_output_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_htlc_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

print("unsigned_htlc_tx: ", unsigned_htlc_tx.hex())



HTLC Output SPK: 0020978f0c6d76e9a391a84ab6385c34d6cc8dcafdc624523ede9241b8cc2211eb76
unsigned_htlc_tx:  0200000001991c932a279fb1878145f93961d703b79ff4e52b5999b139e11b0f3b251b4cc60200000000010000000120a1070000000000220020978f0c6d76e9a391a84ab6385c34d6cc8dcafdc624523ede9241b8cc2211eb7600000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

Just uncomment the lines below to do that.

In [3]:
decoded = node.decoderawtransaction(unsigned_htlc_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "95d43493eae4b20a34f4978deb182a288df2ac6cc64e8266fcfa97897d0d62f1",
  "hash": "95d43493eae4b20a34f4978deb182a288df2ac6cc64e8266fcfa97897d0d62f1",
  "version": 2,
  "size": 94,
  "vsize": 94,
  "weight": 376,
  "locktime": 0,
  "vin": [
    {
      "txid": "c64c1b253b0f1be139b199592be5f49fb703d76139f9458187b19f272a931c99",
      "vout": 2,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 1
    }
  ],
  "vout": [
    {
      "value": "0.00500000",
      "n": 0,
      "scriptPubKey": {
        "asm": "0 978f0c6d76e9a391a84ab6385c34d6cc8dcafdc624523ede9241b8cc2211eb76",
        "desc": "addr(bcrt1qj78scmtkax3er2z2kcu9cdxkejxu4lwxy3frah5jgxuvcgs3admqdd0pwu)#j92lqkdq",
        "hex": "0020978f0c6d76e9a391a84ab6385c34d6cc8dcafdc624523ede9241b8cc2211eb76",
        "address": "bcrt1qj78scmtkax3er2z2kcu9cdxkejxu4lwxy3frah5jgxuvcgs3admqdd0pwu",
        "type": "witness_v0_scripthash"
      }
    }
  ]
}


### Signing the HTLC-Timeout and HTLC-Success Transactions

Segwit transactions have a signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki). According to the sighash type (ALL, NONE, SINGLE, ANYONECANPAY), a transaction digest is generated with a double SHA256 of a serialized subset of the transaction (tx_digest_preimage), and the signature is verified against this digest with a given public key.

#### Creating the Transaction Digest Preimage

In [4]:
hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(alice_offered_htlc_redeemScript)
    + alice_offered_htlc_redeemScript
    + alice_offered_htlc_output_value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print("tx_digest_preimage: ",tx_digest_preimage.hex())

tx_digest_preimage:  020000002fa0db9956056486ab781937dc501561e279d26b4d1735554606900b29d47bda41f758f2e5cc078d3795b4fc0cb60c2d735fa92cc020572bdc982dd2d564d11b991c932a279fb1878145f93961d703b79ff4e52b5999b139e11b0f3b251b4cc6020000008876a9145aab2da71aa3e8e99c72c7b07b04fb3a689447d68763ac672103f0ce4fe88d9e1d79c449ced61ee6cfc9280b3e424a48f952667b7fd73a299aa17c820120876475527c2103f020b85b2f600cdba67ecb33ed2ed09bf1c0a0e67e839a9ca70766049a5f329852ae67a914c51b66bced5e4491001bd702669770dccf44098288ac6851b2756820a107000000000001000000c8537014c8acc06799c874db2786837f66e57129eff5f04fa0741ef6eaa722400000000001000000
